# Interval methods for linear equation 

In [1]:
!pip install -q pyinterval

In [2]:
from interval import interval
from interval import imath
import pandas as pd
import numpy as np
import math

In [3]:
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_midpoint(interv):
    return (interv[0].sup + interv[0].inf) / 2

def interval_to_str(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

def to_table(interv, interv_ext, width, root, tbl):
    tbl.loc[len(tbl)] = [interv, interv_ext, width,root]

def to_table_ext(interv, interv_ext, width, middle, root, tbl):
    tbl.loc[len(tbl)] = [interv, interv_ext, width, middle,root]

## Interval division 

In [4]:
def interval_division(f, start, end, e, tbl):
    x = interval[start, end]
    width = calculate_width(x)
    mid = calculate_midpoint(x)
    Fx = f(x)
    root_exists = 0 in Fx and width < e
    
    #If no 0 in F(x)->no root OR if (0 in F(x) and interval width less than epsilon)->root
    if 0 not in Fx or root_exists :
        to_table_ext(interval_to_str(x), interval_to_str(Fx), width, mid, 'Root' if root_exists else '', tbl)
        return   
        
    tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, mid, '']
    interval_division(f, start, mid, e, tbl)
    interval_division(f, mid, end, e, tbl)

In [5]:
func = lambda x: imath.sin(x-1)*(x-2)*(x-3)
dfunc = lambda x: (x**2-5*x+6)*imath.cos(x-1)-(5-2*x)*imath.sin(x-1)
e = 1e-2
a = 0.9
b = 2.3

In [6]:
interval_division_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Middle point', 'Root'])
interval_division(func, a, b, e, interval_division_result)
interval_division_result

,Interval,Interval extension,Width,Middle point,Root
0,"[0.9000000, 2.3000000]","[-0.6070417, 2.2258194]",1.400000,1.600000,
1,"[0.9000000, 1.6000000]","[-0.2306152, 1.3043241]",0.700000,1.250000,
2,"[0.9000000, 1.2500000]","[-0.2306152, 0.5715031]",0.350000,1.075000,
3,"[0.9000000, 1.0750000]","[-0.2306152, 0.1730876]",0.175000,0.987500,
4,"[0.9000000, 0.9875000]","[-0.2306152, -0.0254700]",0.087500,0.943750,
5,"[0.9875000, 1.0750000]","[-0.0254700, 0.1526810]",0.087500,1.031250,
6,"[0.9875000, 1.0312500]","[-0.0254700, 0.0636664]",0.043750,1.009375,
7,"[0.9875000, 1.0093750]","[-0.0254700, 0.0191027]",0.021875,0.998437,
8,"[0.9875000, 0.9984375]","[-0.0254700, -0.0031323]",0.010937,0.992969,
9,"[0.9984375, 1.0093750]","[-0.0031323, 0.0187937]",0.010937,1.003906,


## Moore's method modification 

In [7]:
def moore(f, df, start, end, e, tbl):
    x = interval([start, end])
    width = calculate_width(x)
    Fx = f(x)

    # If no 0 in F(X) -> no root
    if 0 not in Fx:
        to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
        return
        
    # If interval width less than epsilon -> root
    if width < e:
        to_table(interval_to_str(x), interval_to_str(Fx), width, 'Root', tbl)
        return
        
    # Half if 0 in F'(X)
    x_middle = calculate_midpoint(x)
    dFx = df(x)
    if 0 in dFx:
        to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
        moore(f, df, start, x_middle, e, tbl)
        moore(f, df, x_middle, end, e, tbl)
        return
        
    # Stop if x_i+1 is empty
    f_middle = f(x_middle)
    U = x_middle - f_middle / dFx
    x_next = U & x
    if not x_next:
        to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
        return

    # Continue with narrowed interval
    to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
    moore(f, df, x_next[0].inf, x_next[0].sup, e, tbl)

In [8]:
moore_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Root'])
moore(func, dfunc, a, b, e, moore_result)
moore_result

,Interval,Interval extension,Width,Root
0,"[0.9000000, 2.3000000]","[-0.6070417, 2.2258194]",1.400000,
1,"[0.9000000, 1.6000000]","[-0.2306152, 1.3043241]",0.700000,
2,"[0.9000000, 1.2500000]","[-0.2306152, 0.5715031]",0.350000,
3,"[0.9000000, 1.0750000]","[-0.2306152, 0.1730876]",0.175000,
4,"[0.9960611, 1.0089384]","[-0.0079244, 0.0179824]",0.012877,
5,"[0.9999063, 1.0000695]","[-0.0001875, 0.0001390]",0.000163,Root
6,"[1.0750000, 1.2500000]","[0.0983452, 0.4405337]",0.175000,
7,"[1.2500000, 1.6000000]","[0.1385462, 0.7410932]",0.350000,
8,"[1.6000000, 2.3000000]","[-0.4046944, 0.5395926]",0.700000,
9,"[1.6000000, 1.9500000]","[0.0296437, 0.4555127]",0.350000,


## Hansen's method modification 

In [9]:
def hansen(f, df, start, end, e, tbl):
    x = interval([start, end])
    width = calculate_width(x)
    Fx = f(x)

    # If no 0 in F(X) -> no root
    if 0 not in Fx:
        to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
        return

    # If interval width less than epsilon -> root
    if width < e:
        to_table(interval_to_str(x), interval_to_str(Fx), width, 'Root', tbl)
        return
        
    # Half if 0 in F'(X)
    x_middle = calculate_midpoint(x)
    f_middle = f(x_middle)
    dFx = df(x)
    if f_middle == 0.0:
        to_table(interval_to_str(x), interval_to_str(Fx), width, 'Root?', tbl)
        hansen(f, df, start, x_middle, tbl)
        hansen(f, df, x_middle, end, tbl)
        return

    # Stop if x_i+1 is empty
    U = x_middle - f_middle / dFx
    x_next = U & x
    if not x_next:
        to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
        return

    # Continue with narrowed intervals
    to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
    [hansen(f, df, x.inf, x.sup, e, tbl) for x in x_next]

In [10]:
hansen_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Root'])
hansen(func, dfunc, a, b, e, hansen_result)
hansen_result

,Interval,Interval extension,Width,Root
0,"[0.9000000, 2.3000000]","[-0.6070417, 2.2258194]",1.400000,
1,"[0.9000000, 1.5555241]","[-0.2306152, 1.2182678]",0.655524,
2,"[0.9000000, 1.1548638]","[-0.2306152, 0.3563072]",0.254864,
3,"[0.9280922, 1.0107439]","[-0.1595619, 0.0238605]",0.082652,
4,"[0.9939627, 1.0055862]","[-0.0121840, 0.0112737]",0.011623,
5,"[0.9999936, 1.0000067]","[-0.0000129, 0.0000133]",0.000013,Root
6,"[1.3441439, 1.5555241]","[0.2166167, 0.5727458]",0.211380,
7,"[1.6406772, 2.3000000]","[-0.3929360, 0.4706361]",0.659323,
8,"[1.6406772, 1.9590661]","[0.0254694, 0.3998606]",0.318389,
9,"[1.9767878, 2.3000000]","[-0.2957774, 0.0228855]",0.323212,


## Krawczyk's method modification 

In [11]:
def krawczyk(f, df, start, end, e, tbl):
    x = interval([start, end])
    width = calculate_width(x)
    Fx = f(x)

    # If no 0 in F(X) -> no root
    if 0 not in Fx:
        to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
        return

    # If interval width less than epsilon -> root
    if width < e:
        to_table(interval_to_str(x), interval_to_str(Fx), width, 'Root', tbl)
        return

    # Split interval if 0 in F'(X)
    x_middle = calculate_midpoint(x)
    dFx = df(x)
    if 0 in dFx:
        to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
        krawczyk(f, df, start, x_middle, e, tbl)
        krawczyk(f, df, x_middle, end, e, tbl)
        return

    # Stop if x_i+1 is empty
    dFx_middle = df(x_middle)
    K = x_middle - f(x_middle) / dFx_middle + (1 - dFx / dFx_middle)*(x - x_middle)
    x_next = K & x
    if not x_next:
        to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
        return

    # Display interval if shrunk or not
    to_table(interval_to_str(x), interval_to_str(Fx), width, '', tbl)
    krawczyk(f, df, x_next[0].inf, x_next[0].sup, e, tbl)

In [12]:
krawczyk_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Root'])
krawczyk(func, dfunc, a, b, e, krawczyk_result)
krawczyk_result

,Interval,Interval extension,Width,Root
0,"[0.9000000, 2.3000000]","[-0.6070417, 2.2258194]",1.400000,
1,"[0.9000000, 1.6000000]","[-0.2306152, 1.3043241]",0.700000,
2,"[0.9000000, 1.2500000]","[-0.2306152, 0.5715031]",0.350000,
3,"[0.9000000, 1.0750000]","[-0.2306152, 0.1730876]",0.175000,
4,"[0.9618359, 1.0377099]","[-0.0807338, 0.0797734]",0.075874,
5,"[0.9927271, 1.0072727]","[-0.0147047, 0.0147044]",0.014546,
6,"[0.9997350, 1.0002650]","[-0.0005303, 0.0005303]",0.000530,Root
7,"[1.0750000, 1.2500000]","[0.0983452, 0.4405337]",0.175000,
8,"[1.2500000, 1.6000000]","[0.1385462, 0.7410932]",0.350000,
9,"[1.6000000, 2.3000000]","[-0.4046944, 0.5395926]",0.700000,
